# Solvating and equilibrating a ligand in a box of water

<details>
    <summary><small>▼ Click here for dependency installation instructions</small></summary>
    The simplest way to install dependencies is to use the Interchange examples environment. From the root of the cloned openff-interchange repository:
    
    conda env create --name interchange-examples --file devtools/conda-envs/examples_env.yaml 
    conda activate interchange-examples
    pip install -e .
    cd examples
    jupyter notebook ligand_in_water.ipynb
    
</details>

In [ ]:
import time

import openmm
import openmm.app
from openff.evaluator.protocols import coordinates
from openff.evaluator.utils import packmol
from openff.toolkit import ForceField, Molecule, Topology
from openff.units import unit

from openff.interchange import Interchange

## Construct the topology

In this example we'll construct a topology consisting of one ligand in a box of 1,000 water molecules. OpenFF Evaluator has a [PACKMOL](http://leandro.iqm.unicamp.br/m3g/packmol/home.shtml) wrapper that we will use to generate a reasonable initial configuration. It uses MDTraj's [`Trajectory`](https://www.mdtraj.org/1.9.7/load_functions.html#trajectory-reference) object to store this information; later on we'll move everything to objects provided by the OpenFF Toolkit.

In [ ]:
ligand = Molecule.from_smiles("OC[C@H](O)[C@H]1OC(=O)C(O)=C1O")
ligand.name = ""
water = Molecule.from_smiles("O")
water.name = ""

molecules = [ligand, water]
n_molecules = [1, 1000]
trajectory, _ = packmol.pack_box(
    molecules=molecules,
    number_of_copies=n_molecules,
    mass_density=0.95 * unit.grams / unit.milliliters,
)

There are a few ways to convert the information in this trajectory to an Openff [`Topology`](https://docs.openforcefield.org/projects/toolkit/en/stable/api/generated/openff.toolkit.topology.Topology.html#openff.toolkit.topology.Topology) object. In this case, since we already know how many of which molecules we want, we'll use [`Topology.from_molecules`](https://docs.openforcefield.org/projects/toolkit/en/stable/api/generated/openff.toolkit.topology.Topology.html#openff.toolkit.topology.Topology.from_molecules), which takes a list of `Molecule` objects and assembles them into a `Topology`.

In [ ]:
topology = Topology.from_molecules([ligand, *1000 * [water]])

Next we'll assign the topology's box vectors using the same information on the MDTraj object:

In [ ]:
trajectory.unitcell_vectors * unit.nanometer
vectors = [
    trajectory.unitcell_vectors[0][0],
    trajectory.unitcell_vectors[0][1],
    trajectory.unitcell_vectors[0][2],
] * unit.nanometer
topology.box_vectors = vectors

The ["Sage"](https://openforcefield.org/community/news/general/sage2.0.0-release/) force field line (version 2.x.x) includes TIP3P  parameters for water, so we don't need to use multiple force fields to parametrize this topology.

Note that the "Parsley" (version 1.x.x) line did *not* include TIP3P parameters, so loading in an extra force field was required.

In [ ]:
sage = ForceField("openff-2.0.0.offxml")

From here, we can create an ``Interchange`` object and promptly export it to an [``openmm.System``](http://docs.openmm.org/latest/api-python/generated/openmm.openmm.System.html#openmm.openmm.System):

In [ ]:
if False:
    interchange = Interchange.from_smirnoff(force_field=sage, topology=topology)
    system = interchange.to_openmm(combine_nonbonded_forces=True)
else:
    system = sage.create_openmm_system(topology)

Now, we can prepare everything else that OpenMM needs to run and report a brief equilibration simulation:
* A barostat, since we want to use NPT dynamics to relax the box size toward equilibrium
* An integrator
* A [`Simulation`](http://docs.openmm.org/latest/api-python/generated/openmm.app.simulation.Simulation.html#openmm.app.simulation.Simulation) object, putting it together
* Reporters for the trajectory and simulation data

In [ ]:
barostat = openmm.MonteCarloBarostat(
    1.00 * openmm.unit.bar, 293.15 * openmm.unit.kelvin, 25
)
system.addForce(barostat)

integrator = openmm.LangevinIntegrator(
    300 * openmm.unit.kelvin, 1 / openmm.unit.picosecond, 2 * openmm.unit.femtoseconds
)

simulation = openmm.app.Simulation(openmm_topology, system, integrator)
simulation.context.setPositions(trajectory.openmm_positions(0))
simulation.context.setVelocitiesToTemperature(300 * openmm.unit.kelvin)

pdb_reporter = openmm.app.PDBReporter("trajectory.pdb", 50)
state_data_reporter = openmm.app.StateDataReporter(
    "data.csv", 10, step=True, potentialEnergy=True, temperature=True, density=True
)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(state_data_reporter)

Finally, we can run this simulation. This should take approximately 10-20 seconds on a laptop or small workstation.

In [ ]:
print("Starting simulation")
start_time = time.process_time()

for i in range(5000):
    simulation.step(1)
    if i % 200 == 0:
        print(simulation.context.getState().getPeriodicBoxVectors())

end_time = time.process_time()
print(f"Elapsed time: {(end_time - start_time):.2f} seconds")

## Appendix: visualizing the trajectory

If [NGLView](http://nglviewer.org/nglview/latest/) is installed, we can use it and MDTraj to load and visualize the PDB trajectory:

In [ ]:
import mdtraj
import nglview

In [ ]:
view = nglview.show_mdtraj(mdtraj.load("trajectory.pdb"))
view.add_ball_and_stick("water")
view